<img src="https://placeimg.com/1000/350/nature">

# Script em Python para adequação de nomenclatura botânica a partir do banco de dados da Flora do Brasil 2020

## Motivação

Para quem trabalha com vegetação natural, o uso correto da nomenclatura botânica é essencial, sendo um dos fundamentos para o uso sustentável dos recursos naturais, principalmente em uma país megadiverso como o nosso. 

Neste contexto,  o projeto [Flora do Brasil](http://floradobrasil.jbrj.gov.br/reflora/listaBrasil/PrincipalUC/PrincipalUC.do#CondicaoTaxonCP), do Jardim Botânico do Rio de Janeiro, fruto da colaboração de diversos taxonomistas do Brasil, é uma iniciativa de grande valor, pois reúne informações atuais e corretas das espécies vegetais. 

No site da **Flora do Brasil**, a conferência de nomenclaturas botânicas, como, por exemplo, os nomes aceitos e sinonímias, pode ser realizada de forma bastante intuitiva, a partir da consulta de um formulário online. Porém, quando trabalhamos com listagens grandes, provinientes de **levantamentos florísticos, fitossociológicos e inventários florestais**, este trabalho manual se torna pouco produtivo.

Desta forma, por ser uma demanda frequetemente relacionado às minhas atividades profissionais, resolvi escrever um script e função em linguagem de programação em Python, para automatizar esse processo. 


## Por que Python?

Pela grande comunidade de usuários ativios e por ter um potencial incrível na área de Ciência de Dados, principalmente na aquisição e manipulação de dados não estruturados.

Aqui utilizei a versão do Python disponbilizada pela distribuição [Anaconda](https://www.anaconda.com/), que já vem com várias bibliotecas importantes, como o [Pandas](https://pandas.pydata.org/) e o [Numpy](https://numpy.org/). 

## Etapas

### 1) Carregando pacotes e módulos

In [2]:
import pandas as pd
import difflib
import zipfile

###  2) Importação de arquivos

São necessários dois arquivos:
1. Base original da [Flora do Brasil](http://ipt.jbrj.gov.br/jbrj/resource?r=lista_especies_flora_brasil), disponível de forma pública, na seção de Download, com a descrição  ***Data as a DwC-A file***. Conforme informação do site, este arquivo é atualizado semanalmente. Fazer o download do arquivo na pasta do projeto.
2. Arquivo com relação das espécies a ser corrigida, no formato csv, que também deve estar dentro da pasta do projeto. Arquivo exemplo disponível [aqui](https://1drv.ms/u/s!AuA3UlSxIwNHg61fi4IEylwLuvnvog?e=7qCM8O)

In [3]:
#Importação da base de dados da Flora do Brasil

with zipfile.ZipFile('dwca-lista_especies_flora_brasil-v393.222.zip') as zip:
            with zip.open('taxon.txt') as myZip:
                flora_df = pd.read_csv(myZip, sep='\t', low_memory=False)
                
my_species=pd.read_csv("my_species.csv")

### 3) Verificando importações e análises exploratórias

Os métodos <em>head</em> e <em>tail</em> permitem verificar as primeiras e últimas linhas dos arquivos importados, o que é útil para checar a ocorrência de algum erro na importação.

In [4]:
flora_df.head()

,id,taxonID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,genus,specificEpithet,infraspecificEpithet,taxonRank,scientificNameAuthorship,taxonomicStatus,nomenclaturalStatus,modified,bibliographicCitation,references
0,12,12,NaN,120181.0,NaN,Agaricales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2018-08-10 11:58:06.954,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
1,13,13,NaN,120181.0,NaN,Auriculariales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-09-26 11:01:06.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
2,14,14,NaN,120181.0,NaN,Boletales E.-J.Gilbert,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,NaN,ORDEM,E.-J.Gilbert,NOME_ACEITO,NOME_CORRETO,2019-12-10 16:45:09.658,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
3,15,15,NaN,120181.0,NaN,Geastrales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2015-03-05 17:48:09.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
4,16,16,NaN,120180.0,NaN,Helotiales,NaN,Ascomycota Caval.-Sm.,NaN,NaN,...,NaN,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-03-17 08:17:41.765,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...


In [5]:
flora_df.tail()

,id,taxonID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,genus,specificEpithet,infraspecificEpithet,taxonRank,scientificNameAuthorship,taxonomicStatus,nomenclaturalStatus,modified,bibliographicCitation,references
123498,606941,606941,NaN,113348.0,NaN,Fridericia trichoclada (DC.) Kaehler & L.G. Lo...,NaN,Fridericia Mart.,Taxon 68(4): 765 2019,2019,...,Fridericia,trichoclada,NaN,ESPECIE,(DC.) Kaehler & L.G. Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 16:01:03.511,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
123499,606942,606942,NaN,113938.0,112896.0,Tanaecium dichotomum (Jacq.) L.G.Lohmann,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 65 2019,2019,...,Tanaecium,dichotomum,NaN,ESPECIE,(Jacq.) L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:42:18.643,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
123500,606943,606943,NaN,113938.0,113777.0,Tanaecium parviflorum (Mart. ex DC.) Kaehler &...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,Tanaecium,parviflorum,NaN,ESPECIE,(Mart. ex DC.) Kaehler & L.G.Lohmann,NOME_ACEITO,NaN,2020-01-28 14:50:53.874,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
123501,606944,606944,NaN,113938.0,113712.0,Tanaecium paradoxum (Sandwith) Kaehler & L.G.L...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,Tanaecium,paradoxum,NaN,ESPECIE,(Sandwith) Kaehler & L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:58:07.396,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...
123502,606945,606945,NaN,26542.0,43353.0,Didymopanax calvus (Cham.) Decne. & Planch.,NaN,Didymopanax Decne. & Planch.,NaN,NaN,...,Didymopanax,calvus,NaN,ESPECIE,(Cham.) Decne. & Planch.,NOME_ACEITO,NOME_CORRETO,2020-01-28 17:04:58.733,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...


Em particular, é possível observar que o arquivo importado do site da Flora do Brasil tem 123.503 linhas por 26 colunas

In [6]:
flora_df.shape

(123503, 26)

Para a lista de espécie didática, do [arquivo exemplo](https://1drv.ms/u/s!AuA3UlSxIwNHg61fi4IEylwLuvnvog?e=7qCM8O), temos alguns casos que serão considerados neste processo:
* No índice 0, Acca sellowina escrito de forma incorreta (Aca sloviana);
* No índice 7, uma espécie não indenticada como NI
* Nos índices 10 (<em>Sebastiania commersoniana</em>) e 13 (<em>Gochnatia polymorpha</em>), nomes não aceitos.

In [195]:
my_species

,species
0,Aca sloviana
1,Allophylus edulis
2,Allophylus guaraniticus
3,Annona rugulosa
4,Zanthoxylum rhoifolium
5,Banara tomentosa
6,Blepharocalyx salicifolius
7,NI
8,Campomanesia xanthocarpa
9,Casearia decandra


### 4) Removendo nomes dos autores botânicos das espécies

Na etapa de análise de dados, normalmente trabalha-se com os nomes científicos, sem a presença dos nomes das autoridades botânicas. Por isso, faz sentido removermos os autores dos nomes das espécies no arquivo da Flora do Brasil.

In [7]:
# Primeiramente, criando um nova coluna scientificName1, a partir da remoção dos autores de scientificName
flora_df['scientificName1'] = flora_df.apply(lambda row : row['scientificName'].replace(str(row['scientificNameAuthorship']), ''), axis=1)

# Depois, removendo um "espaço" que ficou residual, no final de 'scientificName1
flora_df['scientificName1'] = flora_df['scientificName1'].str.rstrip()
flora_df

,id,taxonID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,specificEpithet,infraspecificEpithet,taxonRank,scientificNameAuthorship,taxonomicStatus,nomenclaturalStatus,modified,bibliographicCitation,references,scientificName1
0,12,12,NaN,120181.0,NaN,Agaricales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2018-08-10 11:58:06.954,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Agaricales
1,13,13,NaN,120181.0,NaN,Auriculariales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-09-26 11:01:06.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Auriculariales
2,14,14,NaN,120181.0,NaN,Boletales E.-J.Gilbert,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,E.-J.Gilbert,NOME_ACEITO,NOME_CORRETO,2019-12-10 16:45:09.658,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Boletales
3,15,15,NaN,120181.0,NaN,Geastrales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2015-03-05 17:48:09.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Geastrales
4,16,16,NaN,120180.0,NaN,Helotiales,NaN,Ascomycota Caval.-Sm.,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-03-17 08:17:41.765,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Helotiales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123498,606941,606941,NaN,113348.0,NaN,Fridericia trichoclada (DC.) Kaehler & L.G. Lo...,NaN,Fridericia Mart.,Taxon 68(4): 765 2019,2019,...,trichoclada,NaN,ESPECIE,(DC.) Kaehler & L.G. Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 16:01:03.511,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Fridericia trichoclada
123499,606942,606942,NaN,113938.0,112896.0,Tanaecium dichotomum (Jacq.) L.G.Lohmann,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 65 2019,2019,...,dichotomum,NaN,ESPECIE,(Jacq.) L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:42:18.643,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium dichotomum
123500,606943,606943,NaN,113938.0,113777.0,Tanaecium parviflorum (Mart. ex DC.) Kaehler &...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,parviflorum,NaN,ESPECIE,(Mart. ex DC.) Kaehler & L.G.Lohmann,NOME_ACEITO,NaN,2020-01-28 14:50:53.874,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium parviflorum
123501,606944,606944,NaN,113938.0,113712.0,Tanaecium paradoxum (Sandwith) Kaehler & L.G.L...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,paradoxum,NaN,ESPECIE,(Sandwith) Kaehler & L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:58:07.396,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium paradoxum


### 5) Correção de erros tipográficos na lista florística, a partir da coluna scientificName1

Para a correção dos erros tipográficos a partir da grafia correta encontrada na **Flora do Brasil**, utilizamos o módulo [diffilib](https://docs.python.org/2/library/difflib.html), que, conforme a documentação, basea-se no o algorítimo [Gestalt Pattern Matching](https://en.wikipedia.org/wiki/Gestalt_Pattern_Matching). 

Deve-se ter atenção, pois este passo é computacionalmente intensivo, de forma que pode ser bem lento no caso de listagens florísticas extensas. Além disso, é importante uma análise <em>a posteriori</em> dos resultados desta etapa, para verificar eventuais incoerências na correção dos nomes científicos.

In [8]:
possibilities = flora_df['scientificName1']
species_typo_corrected = my_species['species'].apply(lambda row: difflib.get_close_matches(row, possibilities, n=1) [0] if difflib.get_close_matches(row, possibilities) else row)


Como resultado, podemos observar que Aca sloviana ficou devidamente corrigida para <em>Acca sellowiana</em>. 

Importante ressaltar que as espécies sem referência com similaridade no arquivo da Flora do Brasil, como é o caso de NI, a nomenclatura da mesma é mantida.

In [9]:
species_typo_corrected.df = pd.DataFrame(species_typo_corrected)
species_typo_corrected.columns=['species']
species_typo_corrected.df

,species
0,Acca sellowiana
1,Allophylus edulis
2,Allophylus guaraniticus
3,Annona rugulosa
4,Zanthoxylum rhoifolium
5,Banara tomentosa
6,Blepharocalyx salicifolius
7,NI
8,Campomanesia xanthocarpa
9,Casearia decandra


### 6) Correção de sinonías botânicas

Com os eventuais erros tipográficos corrigidos, agora podemos fazer a verificação da existência de nomes não aceitos e sinônimos, com a subsequente correção.

In [115]:
species_syno_corrected=[]

for species in species_typo_corrected:
    
    line = flora_df[flora_df['scientificName1'] == species]
    accepted_name = line["acceptedNameUsage"]
    species_syno_corrected.append(list(accepted_name))
    species_syno_corrected_df=pd.DataFrame(species_syno_corrected) 
    final_table=pd.concat([species_typo_corrected, species_syno_corrected_df],  axis = 1) 
    final_table.columns =['Species', 'Correction']
    final_table['Correction'] = final_table.apply(lambda x: x['Species'] if pd.isnull(x['Correction']) else x['Correction'], axis=1)
    final_table['Correction'] = final_table['Correction'].str.extract(r'(^\w*\s*\w*)')
    
    
    

final_table

,Species,Correction
0,Acca sellowiana,Acca sellowiana
1,Allophylus edulis,Allophylus edulis
2,Allophylus guaraniticus,Allophylus guaraniticus
3,Annona rugulosa,Annona rugulosa
4,Zanthoxylum rhoifolium,Zanthoxylum rhoifolium
5,Banara tomentosa,Banara tomentosa
6,Blepharocalyx salicifolius,Blepharocalyx salicifolius
7,NI,NI
8,Campomanesia xanthocarpa,Campomanesia xanthocarpa
9,Casearia decandra,Casearia decandra


Observe que <em>Gochnatia polymorpha</em> foi atualizado para <em>Gymnanthes klotzschiana</em>. Por sua vez, <em>Gochnatia polymorpha</em> atualizado para <em>Moquiniastrum polymorphum</em>.

### 7) Como etapa final, a criação das funções "flora_clean", "flora_typo_correct"  e "flora_accepted_names"

A partir dos scripts acima, podemos definir funções específicas para serem aproveitas em projetos de manipulação e limpeza de dados botânicos

In [133]:
# Funcao flora_clean, para limpeza dos dados originais, para posterior aplicação nas funções subsequentes

def flora_clean (flora_df):
    ''' flora_df = arquivo importado a partir do banco de dados da Flora do Brasil'''
    
    flora_df['scientificName1'] = flora_df.apply(lambda row : row['scientificName'].replace(str(row['scientificNameAuthorship']), ''), axis=1)
    flora_df['scientificName1'] = flora_df['scientificName1'].str.rstrip()
    return flora_df

# Função flora_typo_correct, para correcao de erros tipográficos

def flora_typo_correct (my_species,flora_clean):
   
    ''' 
    my_species = arquivo importado a partir da listagem florística a ser corrigida
    flora_clean = Arquivo da Flora do Brasil limpo, após a aplicação da função flora_clean
    
    '''
    
    possibilities = flora_clean['scientificName1']
    species_typo_corrected = my_species['species'].apply(lambda row: difflib.get_close_matches(row, possibilities, n=1) [0] if difflib.get_close_matches(row, possibilities) else row)
    species_typo_corrected.df = pd.DataFrame(species_typo_corrected)
    species_typo_corrected.columns=['species']
    species_typo_corrected.df
    return species_typo_corrected.df


def flora_accepted_names(my_species_clean, data_clean):
    
    ''' 
    my_clean_species = lista florística corrigida, a partir da aplicação de flora_typo_correct
    flora_clean = Arquivo da Flora do Brasil limpo, após a aplicação da função flora_clean
    
    '''
    
    species_syno_corrected=[]
    
    for species in my_species_clean['species']:
        
        line = data_clean[data_clean['scientificName1'] == species]
        accepted_name = line["acceptedNameUsage"]
        species_syno_corrected.append(list(accepted_name))
        species_syno_corrected_df=pd.DataFrame(species_syno_corrected) 
        final_table=pd.concat([my_species_clean, species_syno_corrected_df],  axis = 1) 
        final_table.columns =['Species', 'Correction']
        final_table['Correction'] = final_table.apply(lambda x: x['Species'] if pd.isnull(x['Correction']) else x['Correction'], axis=1)
        final_table['Correction'] = final_table['Correction'].str.extract(r'(^\w*\s*\w*)')
    return final_table


### 8) Aplicando as funções

In [44]:
#Importação da base de dados da Flora do Brasil

with zipfile.ZipFile('dwca-lista_especies_flora_brasil-v393.222.zip') as zip:
            with zip.open('taxon.txt') as myZip:
                data_df = pd.read_csv(myZip, sep='\t', low_memory=False)
                
minha_lista=pd.read_csv("my_species.csv")

In [45]:
#Aplicação da função flora_clean

data_clean=flora_clean(data_df)

data_clean

,id,taxonID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,scientificName,acceptedNameUsage,parentNameUsage,namePublishedIn,namePublishedInYear,...,specificEpithet,infraspecificEpithet,taxonRank,scientificNameAuthorship,taxonomicStatus,nomenclaturalStatus,modified,bibliographicCitation,references,scientificName1
0,12,12,NaN,120181.0,NaN,Agaricales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2018-08-10 11:58:06.954,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Agaricales
1,13,13,NaN,120181.0,NaN,Auriculariales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-09-26 11:01:06.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Auriculariales
2,14,14,NaN,120181.0,NaN,Boletales E.-J.Gilbert,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,E.-J.Gilbert,NOME_ACEITO,NOME_CORRETO,2019-12-10 16:45:09.658,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Boletales
3,15,15,NaN,120181.0,NaN,Geastrales,NaN,Basidiomycota,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2015-03-05 17:48:09.918,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Geastrales
4,16,16,NaN,120180.0,NaN,Helotiales,NaN,Ascomycota Caval.-Sm.,NaN,NaN,...,NaN,NaN,ORDEM,NaN,NOME_ACEITO,NOME_CORRETO,2014-03-17 08:17:41.765,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Helotiales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123498,606941,606941,NaN,113348.0,NaN,Fridericia trichoclada (DC.) Kaehler & L.G. Lo...,NaN,Fridericia Mart.,Taxon 68(4): 765 2019,2019,...,trichoclada,NaN,ESPECIE,(DC.) Kaehler & L.G. Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 16:01:03.511,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Fridericia trichoclada
123499,606942,606942,NaN,113938.0,112896.0,Tanaecium dichotomum (Jacq.) L.G.Lohmann,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 65 2019,2019,...,dichotomum,NaN,ESPECIE,(Jacq.) L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:42:18.643,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium dichotomum
123500,606943,606943,NaN,113938.0,113777.0,Tanaecium parviflorum (Mart. ex DC.) Kaehler &...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,parviflorum,NaN,ESPECIE,(Mart. ex DC.) Kaehler & L.G.Lohmann,NOME_ACEITO,NaN,2020-01-28 14:50:53.874,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium parviflorum
123501,606944,606944,NaN,113938.0,113712.0,Tanaecium paradoxum (Sandwith) Kaehler & L.G.L...,NaN,Tanaecium Sw. emend L.G.Lohmann,Taxon 68(4): 765 2019,2019,...,paradoxum,NaN,ESPECIE,(Sandwith) Kaehler & L.G.Lohmann,NOME_ACEITO,NOME_CORRETO,2020-01-28 14:58:07.396,Flora do Brasil 2020 em construção. Jardim Bot...,http://reflora.jbrj.gov.br/reflora/listaBrasil...,Tanaecium paradoxum


In [50]:
#Aplicação da função flora_typo_correct

my_species_clean=flora_typo_correct(minha_lista, data_clean)

In [51]:
my_species_clean

,species
0,Acca sellowiana
1,Allophylus edulis
2,Allophylus guaraniticus
3,Annona rugulosa
4,Zanthoxylum rhoifolium
5,Banara tomentosa
6,Blepharocalyx salicifolius
7,NI
8,Campomanesia xanthocarpa
9,Casearia decandra


In [134]:
#Aplicação da função flora_typo_correct

flora_accepted_names(my_species_clean, data_clean)

,Species,Correction
0,Acca sellowiana,Acca sellowiana
1,Allophylus edulis,Allophylus edulis
2,Allophylus guaraniticus,Allophylus guaraniticus
3,Annona rugulosa,Annona rugulosa
4,Zanthoxylum rhoifolium,Zanthoxylum rhoifolium
5,Banara tomentosa,Banara tomentosa
6,Blepharocalyx salicifolius,Blepharocalyx salicifolius
7,NI,NI
8,Campomanesia xanthocarpa,Campomanesia xanthocarpa
9,Casearia decandra,Casearia decandra


## Considerações finais

A partir do script em Python apresentado, foi possível explorar o banco de dados da ***Flora do Brasil***, para automatizar a checagem das nomenclaturas botânicas, que uma atividade crítica em análises de dados vegetacionais.  


Como etapas seguintes, pretendo melhorar performance do scripts, para que rode mais rápido, e criar uma interface web por meio do [sreamlit](https://www.streamlit.io/)

## Referência
Brazil Flora G (2020): Brazilian Flora 2020 project - Projeto Flora do Brasil 2020. v393.222. Instituto de Pesquisas Jardim Botanico do Rio de Janeiro. Dataset/Checklist. doi:10.15468/1mtkaw